In [11]:
!pip install yfinance
!pip install konlpy
!pip install exchange_calendars
!pip install schedule

In [12]:
# -*- coding: utf-8 -*-
### import ###
import requests
import pandas as pd
import math
from tqdm import tqdm
import yfinance as yf
import exchange_calendars as ecals

from bs4 import BeautifulSoup
import re
import time
import itertools
import csv
import datetime
import schedule
import konlpy
from konlpy.tag import Okt
import numpy as np
from tqdm import tqdm
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import db

In [13]:
### 모델 학습코드 ###
train_data = pd.read_csv("./train.csv")

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

okt = Okt()
X_train = []
for sentence in train_data['title']:
  temp_X =[]
  temp_X = okt.morphs(str(sentence), stem=True) #토큰화
  temp_X = [word for word in temp_X if not word in stopwords] #안쓰는 말 제거
  X_train.append(temp_X)
  
max_words = 35000
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

y_train = []
for i in range(len(train_data['label'])):
  if train_data['label'].iloc[i]==1:
    y_train.append([0,0,1])
  elif train_data['label'].iloc[i]==0:
     y_train.append([0,1,0]) 
  elif train_data['label'].iloc[i]== -1:
     y_train.append([1,0,0])
y_train = np.array(y_train) 

max_len = 20
X_train = pad_sequences(X_train, maxlen=max_len)
model = Sequential()
model.add(Embedding(max_words, 100))
model.add(LSTM(128))
model.add(Dense(3, activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.1)

Epoch 1/10
504/504 [==============================] - 21s 38ms/step - loss: 0.5397 - accuracy: 0.7820 - val_loss: 0.2693 - val_accuracy: 0.9231
Epoch 2/10
504/504 [==============================] - 18s 36ms/step - loss: 0.0778 - accuracy: 0.9761 - val_loss: 0.2500 - val_accuracy: 0.9428
Epoch 3/10
504/504 [==============================] - 18s 36ms/step - loss: 0.0090 - accuracy: 0.9976 - val_loss: 0.3550 - val_accuracy: 0.9267
Epoch 4/10
504/504 [==============================] - 18s 35ms/step - loss: 0.0030 - accuracy: 0.9996 - val_loss: 0.8873 - val_accuracy: 0.8712
Epoch 5/10
504/504 [==============================] - 18s 35ms/step - loss: 0.0084 - accuracy: 0.9986 - val_loss: 0.3360 - val_accuracy: 0.9392
Epoch 6/10
504/504 [==============================] - 18s 36ms/step - loss: 2.3845e-04 - accuracy: 1.0000 - val_loss: 0.3759 - val_accuracy: 0.9428
Epoch 7/10
504/504 [==============================] - 17s 35ms/step - loss: 6.9555e-05 - accuracy: 1.0000 - val_loss: 0.4029 - val_a

In [14]:
### 파이어베이스 연동 ###
if not firebase_admin._apps:
    cred = credentials.Certificate('./firebase_key.json')
    default_app = firebase_admin.initialize_app(cred)

db = firestore.client()

In [15]:
# 1번 실행
### 날짜 반영 ###
# 만약 장중이 아니면 어제, 그저께로 쿼링해야 값들 가져올 수 있음
# 장중이면 오늘, 어제로 쿼링해야 실시간 반영 가능
XKRX = ecals.get_calendar("XKRX") # 한국 코드

current = datetime.datetime.now()
corr_current = current - datetime.timedelta(hours=9)

if(XKRX.is_trading_minute(corr_current.strftime("%Y-%m-%d %H:%M"))):
    previous = datetime.date.today().strftime("%Y-%m-%d")
    time_previous = datetime.date.today().strftime("%Y-%m-%d %H:%M")
    pre_previous = XKRX.previous_session(previous).strftime("%Y-%m-%d")
else:
    previous = XKRX.previous_minute(corr_current.strftime("%Y-%m-%d %H:%M")).strftime("%Y-%m-%d")
    time_previous = XKRX.previous_minute(corr_current.strftime("%Y-%m-%d %H:%M")) + datetime.timedelta(hours=9)
    pre_previous = XKRX.previous_session(previous).strftime("%Y-%m-%d")
    time_previous = str(time_previous.strftime("%Y-%m-%d %H:%M"))

In [24]:
# 1번 실행
# 주식 이름, 주식 코드, 시가총액 가져오는 함수
# 공공데이터포털 - 금융위원회_주식시세정보 api 켜야함

# 공공데이터포털 api 주소(Without param)
api_service_key_stock = [
    "RXhGWArdgsytKaKf0g%2FWxNuo27wXxg4iChLUs9ePc39VvneddFbQ9v9ZXCDWJkdFbhqCvbw9kdMGy%2F%2Bv3it50A%3D%3D",
    "bqvyeN8k%2B8%2BfRLf7p4CNQsUIEL%2BRb4b2YR08MD10RDv3BxHugq6bR1wFEAo8hTau3XgiLcA7bEBoclnMdyBfNQ%3D%3D",
    "zUgkw3obrruAXAW6kZrJnIpK8UUBIrwXrfroSgoDS7NUlSB%2BDz94OTIkkWeP0V%2BzOz81JVtW84bqh1y0HpzcUg%3D%3D",
    "w9Ra19Zqn3%2BLgg2zHoRiZa8zZPdSCXSgFgrgFGUkaYqqQRD6BVKMsUgiRyJqeEuG1pQ86vSioq03IRarAve7sg%3D%3D",
]  # service api key

code_list = []

# 종목 이름 가져오는 코드
def getStockCode(market, code_list):
    """
    market: 상장구분 (11=유가증권, 12=코스닥, 13=K-OTC, 14=코넥스, 50=기타비상장)
    """
    url = f"https://api.odcloud.kr/api/GetStockSecuritiesInfoService/v1/getStockPriceInfo?"
    stock_code = 0
    while True:
        api_decode_key_stock = requests.utils.unquote(
            api_service_key_stock[stock_code], encoding="utf-8"
        )

        params = {
            "serviceKey": api_decode_key_stock,
            "mrktCls": market,
            "numOfRows": 1000,
            "beginBasDt":pre_previous.replace("-", "")
        }

        response = requests.get(url, params=params)
        if(response.status_code != 200):
            print(response.status_code)
            stock_code += 1
            continue
        xml = BeautifulSoup(response.text, "lxml")
        items = xml.find("items")
        item_list = []
        for item in items:
            try:
                item_dict = {
                    "stockName": item.find("itmsnm").text.strip(),
                    "stockCode": item.find("srtncd").text.strip(),
                    "marketCap": item.find("mrkttotamt").text.strip(),
                }
                code_list.append(item.find("srtncd").text.strip() + ".KS")
            except AttributeError:
                continue
            item_list.append(item_dict)

        return item_list

# 기사 제목 전처리 함수
def text_clean(inputString):
    # inputString = inputString.replace("<b>","").replace("</b>","") # html 태그 제거  ## <b> <b/>
    inputString = re.sub(r'\<[^)]*\>', '', inputString, 0).strip() # <> 안의 내용 제거  ## html태그 + 종목명
    inputString = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString) # 특수문자 제거
    inputString = inputString.replace("&quot;"," ").replace("amp;","").replace("&gt; "," ").replace("&lt;"," ")
    inputString = ' '.join(inputString.split())
    
    return inputString

# 기사 날짜 전처리 함수
def formatting_date(date):
    format ='%a, %d %b %Y %H:%M:%S %z'
    date = datetime.datetime.strptime(date, format) # str to datetime
    date = date.strftime("%Y-%m-%d %H:%M:%S") # changing datetime format
    
    return date

# Naver client key
# client_id= "4NnYXQRzNVwTEO2_rwpd"
# client_secret = "mZP8JBDOBK"

# Naver clint key by sj
client_id = "8Sbpzhlz4LPc0MPsiaOO"
client_secret = "HPqxX8HZfG"

# 네이버 api 함수
def api_search(tuple_list, stock):
    print("api_search  ",stock)
    url = 'https://openapi.naver.com/v1/search/news.json' 
    header = {'X-Naver-Client-Id':client_id, 'X-Naver-Client-Secret':client_secret} 
    param = {'query':stock, 'display':1, 'start':1, 'sort':'date'} 
    # query     : 검색할 단어
    # display   : 검색 출력 건수 (기본 10 / 최대 100)
    # start     : 검색 시작 위치 (기본 1  / 최대 1000)
    # sort      : 정렬순서      (기본 sim : 유사도 / date : 날짜)
    res = requests.get(url, params=param, headers=header)

    pov_or_neg = 0 #긍부정 라벨링 값

    if res.status_code == 200:
        temp = res.json()

        # for index, item in enumerate(temp['items']):
        #     print(index+1, item['title'], item['link'], item['description'],item['pubDate'])

        # TODO
        for dict in temp['items']:
            title  = text_clean(dict['title'])

            #학습데이터를 통해서 라벨링
            X_test=[]
            temp_X = okt.morphs(str(title), stem=True) #토큰화
            temp_X = [word for word in temp_X if not word in stopwords] #안쓰는 말 제거
            X_test.append(temp_X)
            X_test = tokenizer.texts_to_sequences(X_test)
            predict= model.predict(X_test)
            
            # 호악재 예측값 저장
            pov_or_neg = np.argmax(predict,axis=1)[0]

            date = formatting_date(dict['pubDate'])


            # 쿼링 코드
            #
            # 마지막 temp3 부분을 올라갈 문서의 제목으로 바꿔야함.
            news_temp = db.collection(u'test_승준').document(stock).collection(u'news').document(u'temp3')
            news_temp.set({
                u'date': date,
                u'title': dict['title'],
                u'label': str(pov_or_neg),
                u'url': str(dict['originallink'])
            })

            tuple_list.append((stock ,title ,dict['originallink'] ,date ,pov_or_neg))
                # print(stock ,title ,dict['originallink'] ,date ,pov_or_neg)
    else:
        print("Error Code:" + str(res.status_code)+" Stock name is "+ str(stock))


# 종목들 전역변수로 가져오기
company=[]
def get_companylist():
    item_list = getStockCode("KOSPI", code_list)
    item_list.append({
        "stockName": "코스피",
        "stockCode": "^KS11",
        "marketCap": 0,
        "updatedTime" : time_previous,
        })
    item_list.append({
        "stockName": "코스닥",
        "stockCode": "^KQ11",
        "marketCap": 0,
    })
    code_list.append("^KS11")
    code_list.append("^KQ11")
    global company
    company=item_list
    #company = list(itertools.chain.from_iterable(item_list))


def run(reset):
        # 계산된 횟수만 실행
        print("run")
        while reset:
            print("남은횟수: ", reset)
            start = time.time()
            tuple_list=[]
            tuple_list.append(("stock" ,"title" ,"url" ,"date" ,"pov_or_neg"))

            for stock in tqdm(company):
                api_search(tuple_list, stock["stockName"])

            # 쿼링 포함 15분 제한
            end = time.time()
            print(end-start)
            rest_time = 900 - (end-start)
            time.sleep(rest_time) 

            reset -= 1


    # #tuple to csv 저장
    # with open('news.csv', 'w') as f:
    #     writer = csv.writer(f , lineterminator='\n')
    #     for tup in tuple_list:
    #         writer.writerow(tup)


# 8시 20분에 주식가져옴
schedule.every().day.at("08:20").do(get_companylist)
# 8시 30분에 코드 실행
schedule.every().day.at("08:30").do(run)



#시간제한없는거랑
#중복제거
#어떤 형식으로 나오는지 csv만들고
#쿼링
get_companylist()

In [31]:
company=[]
code_list = []
get_companylist()
len(code_list)

1002

In [26]:


if __name__ == "__main__":
    print("start")
    # 주식 종목을 global로 설정
    get_companylist()
    now = datetime.datetime.now()
    time_now = datetime.timedelta(hours= now.hour , minutes=now.minute)
    time_start = datetime.timedelta(hours= 8, minutes=30)
    time_end = datetime.timedelta(hours= 17, minutes=30)

    run_time = 1
    run(run_time)


start
run
남은횟수:  1


  0%|          | 0/1002 [00:00<?, ?it/s]

api_search   동화약품


  0%|          | 1/1002 [00:02<34:35,  2.07s/it]

api_search   KR모터스


...

api_search   삼영무역


 16%|█▌        | 161/1002 [03:36<18:42,  1.33s/it]

api_search   SUN&L


 16%|█▌        | 161/1002 [03:37<18:57,  1.35s/it]


KeyError: ignored

In [ ]:
st = time.time()
# 여기서부터는 시간별로 실행 - 실시간 코드
# start에는 장이 열리는 날 - 하루가 들어가야 함
data = yf.download(code_list, start=pre_previous)

# item_list에 각 필드들 쿼링
for j in tqdm(company):
    stockCode = j["stockCode"]
    if j["stockCode"] != "^KS11" and j["stockCode"] != "^KQ11" :
        stockCode += ".KS"
        
    j["stockPrice"] = float(data["Close"][stockCode][previous])
    j["stockLowPrice"] = float(data["Low"][stockCode][previous])
    j["stockHighPrice"] = float(data["High"][stockCode][previous])
    j["stockVolume"] = float(data["Volume"][stockCode][previous])
    j["stockOpenPrice"] = float(data["Open"][stockCode][previous])
    j["stockClosingPrice"] = float(data["Close"][stockCode][pre_previous])
    j['stockChange'] = float(data['Close'][stockCode][previous] - data['Close'][stockCode][pre_previous])
    j['stockPerChange'] = float((data['Close'][stockCode][previous] - data['Close'][stockCode][pre_previous]) / data['Close'][stockCode][pre_previous] * 100)
    j['DayNewsCount'] = 0
    j['TimeNewsCount'] = 0
    j["TimePerPositiveNewsCount"] = 0
    j["TimePerNegativeNewsCount"] = 0

# 상장폐지 / 거래중지 종목 리스트에서 제거
for i in company:
    if math.isnan(i['stockPrice']) or math.isnan(i['stockLowPrice']) or math.isnan(i['stockHighPrice']) or math.isnan(i['stockVolume']) or math.isnan(i['stockOpenPrice'])or math.isnan(i['stockClosingPrice'])or math.isnan(i['stockChange'])or math.isnan(i['stockPerChange']):
        company.remove(i)

# round 처리로 2자리수까지 보여짐
# nan 처리 해주고 나서 돌려야해서 필요한 코드
for j in tqdm(company):
    stockCode = j["stockCode"]
    if j["stockCode"] != "^KS11" and j["stockCode"] != "^KQ11" :
        stockCode += ".KS"
    j["stockPrice"] = float(round(data["Close"][stockCode][previous], 2))
    j["stockLowPrice"] = float(round(data["Low"][stockCode][previous], 2))
    j["stockHighPrice"] = float(round(data["High"][stockCode][previous], 2))
    j["stockVolume"] = float(round(data["Volume"][stockCode][previous], 2))
    j["stockOpenPrice"] = float(round(data["Open"][stockCode][previous], 2))
    j["stockClosingPrice"] = float(round(data["Close"][stockCode][pre_previous], 2))
    j['stockChange'] = float(round(data['Close'][stockCode][previous] - data['Close'][stockCode][pre_previous], 2))
    j['stockPerChange'] = float(round((data['Close'][stockCode][previous] - data['Close'][stockCode][pre_previous]) / data['Close'][stockCode][pre_previous] * 100, 2))

print(time.time()-st)

[**********************49%                       ]  462 of 942 completed

In [ ]:
# Firebase에 multithreading 이용해서 쿼링
# 3분 30초 -> 20초

db = firestore.client()

from multiprocessing.dummy import Pool as ThreadPool
def AddFirebase(item):
    db.collection(u"stock_API2").document(item["stockName"]).set(item)

pool = ThreadPool(10)

for _ in tqdm(pool.imap_unordered(AddFirebase, item_list), total=len(item_list)):
    pass

pool.close() 
pool.join()

In [ ]:

def make_a_first_endPoint():
    stock_count_list={}

    for i in item_list:
        stock = i['stockName']
        stock_count_list[stock] = {'DayNewsCount':0, 'TimeNewsCount':0,"TimePerPositiveNewsCount" : 0,"TimePerNegativeNewsCount" :0,'endPoint':"temp"}       

        url = 'https://openapi.naver.com/v1/search/news.json' 
        header = {'X-Naver-Client-Id':client_id, 'X-Naver-Client-Secret':client_secret} 
        param = {'query':stock, 'display':1, 'start':1, 'sort':'date'} 

        res = requests.get(url, params=param, headers=header)

        if res.status_code == 200:
            temp = res.json()
            stock_count_list[stock]['endPoint'] = temp['items'][0]['title']
        else: print(stock, "err")
make_a_first_endPoint()

In [ ]:
stock = '삼성전자'
stock_count_list[stock] = {'DayNewsCount':0, 'TimeNewsCount':0,"TimePerPositiveNewsCount" : 0,"TimePerNegativeNewsCount" :0,'endPoint':"temp"}       

url = 'https://openapi.naver.com/v1/search/news.json' 
header = {'X-Naver-Client-Id':client_id, 'X-Naver-Client-Secret':client_secret} 
param = {'query':stock, 'display':1, 'start':1, 'sort':'date'} 

res = requests.get(url, params=param, headers=header)

if res.status_code == 200:
    temp = res.json()
    print(temp['items'][0]['title'])
    stock_count_list[stock]['endPoint'] = temp['items'][0]['title']
else: print(stock, "err")

In [ ]:
stock_count_list[stock]['endPoint']

In [ ]:
#########################     make a first endPoint
tuple_list=[]

url = 'https://openapi.naver.com/v1/search/news.json' 
header = {'X-Naver-Client-Id':client_id, 'X-Naver-Client-Secret':client_secret} 
param = {'query':stock, 'display':1, 'start':1, 'sort':'date'} 
# query     : 검색할 단어
# display   : 검색 출력 건수 (기본 10 / 최대 100)
# start     : 검색 시작 위치 (기본 1  / 최대 1000)
# sort      : 정렬순서      (기본 sim : 유사도 / date : 날짜)
res = requests.get(url, params=param, headers=header)

pov_or_neg = 0 #긍부정 라벨링 값
a = ""
if res.status_code == 200:
    temp = res.json()

    # for index, item in enumerate(temp['items']):
    #     print(index+1, item['title'], item['link'], item['description'],item['pubDate'])

    # TODO
    
    for index, dict in enumerate(temp['items']):
        title  = text_clean(dict['title'])
        if(dict['title'] == stock_count_list['카카오']['endPoint']):
            break
        
        print(text_clean(dict['title']), formatting_date(dict['pubDate']))
        

        #학습데이터를 통해서 라벨링
        X_test=[]
        temp_X = okt.morphs(str(title), stem=True) #토큰화
        temp_X = [word for word in temp_X if not word in stopwords] #안쓰는 말 제거
        X_test.append(temp_X)
        X_test = tokenizer.texts_to_sequences(X_test)
        predict= model.predict(X_test)
        
        # 호악재 예측값 저장
        pov_or_neg = np.argmax(predict,axis=1)[0]

        date = formatting_date(dict['pubDate'])


        # # 쿼링 코드
        # #
        # # 마지막 temp3 부분을 올라갈 문서의 제목으로 바꿔야함.
        # news_temp = db.collection(u'test_승준').document(stock).collection(u'news').document(str(index+1))
        # news_temp.set({
        #     u'date': date,
        #     u'title': dict['title'],
        #     u'label': str(pov_or_neg),
        #     u'url': str(dict['originallink'])
        # })

        tuple_list.append((stock ,title ,dict['originallink'] ,date ,pov_or_neg))
    stock_count_list['카카오']['endPoint'] = temp['items'][0]['title']
else:
    print("Error Code:" + str(res.status_code)+" Stock name is "+ str(stock))

print("endPoint: ",stock_count_list['카카오']['endPoint'])

In [ ]:

for index, dict in enumerate(temp['items']):
    title  = text_clean(dict['title'])

    
    print(text_clean(dict['title']), formatting_date(dict['pubDate']))

In [ ]:
stock_count_list={}
for i in item_list:
    if i['stockName'] not in stock_count_list:
        stock_count_list[i['stockName']] = {'DayNewsCount':0, 'TimeNewsCount':0,"TimePerPositiveNewsCount" : 0,"TimePerNegativeNewsCount" :0,'endPoint':"temp"}       

In [ ]:
stock_count_list['카카오']['endPoint']

In [ ]:
temp_end = a['items'][0]['title']
for dict in a['items']:
    if(text_clean(dict['title']) == stock_count_list['카카오']['endPoint']):
        break
    
    print(text_clean(dict['title']), formatting_date(dict['pubDate']))
stock_count_list['카카오']['endPoint'] = temp_end

In [ ]:
temp_end

In [ ]:
for i in item_list:
    print(i['stockName'])